# Phase 3: Volatility Regime Exploration

**Objective**: Prove that precious metals exhibit distinct volatility regimes that align with macro stress periods.

### 1️⃣ Load 1H Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

# Add root to path to import local modules
sys.path.append(os.path.abspath('..'))

from features.volatility import rolling_volatility
from features.regimes import volatility_regimes

# Load 1H data (XAUUSD)
df = pd.read_csv("../data/processed/XAUUSD_1H.csv", parse_dates=["Datetime"])
df = df.rename(columns={"Datetime": "timestamp"})
df = df.set_index("timestamp")

print("Data Loaded:")
display(df.head())

### 2️⃣ Compute Volatility + Regimes

In [ ]:
df["vol"] = rolling_volatility(df)
df["regime"] = volatility_regimes(df["vol"])

print("Regime Distribution:")
print(df["regime"].value_counts())

### 3️⃣ Plot Volatility with Regime Coloring

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True)
ax1.plot(df.index, df["Close"], color="black", lw=1, label="XAUUSD Close")
colors = {"LOW_VOL": "green", "MID_VOL": "yellow", "HIGH_VOL": "red"}
for regime, color in colors.items():
    mask = df["regime"] == regime
    ax1.fill_between(df.index, df["Close"].min(), df["Close"].max(), where=mask, color=color, alpha=0.15, label=f"{regime} Regime")
ax1.set_title("XAUUSD Price with Volatility Regime Overlays")
ax1.set_ylabel("Price")
handles, labels = ax1.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax1.legend(by_label.values(), by_label.keys())
ax2.plot(df.index, df["vol"], color="blue", lw=1, label="Rolling Volatility")
ax2.set_title("Log-Return Rolling Volatility (w=30)")
ax2.set_ylabel("Volatility")
plt.tight_layout()
plt.show()

# Phase 4: Baseline Strategy Validation

**Objective**: Establish a static momentum control group to act as a benchmark for comparison.

In [ ]:
from strategies.baseline import baseline_strategy
from evaluation.metrics import strategy_returns

# Apply Baseline Strategy
df_baseline = baseline_strategy(df, lookback=20)
df_baseline["strat_ret"] = strategy_returns(df_baseline)
df_baseline["cum_ret"] = df_baseline["strat_ret"].cumsum().apply(np.exp)

display(df_baseline[["momentum", "position", "strat_ret", "cum_ret"]].tail())

### 📊 Baseline Results Visualization

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(df_baseline.index, df_baseline["cum_ret"], label="Baseline Momentum (Control)", color="purple")
plt.title("Baseline Strategy Cumulative Returns (Control Group)")
plt.ylabel("Cumulative Return (Multiplier)")
plt.axhline(1, color="black", linestyle="--", alpha=0.5)
plt.legend()
plt.grid(alpha=0.3)
plt.show()
print("Commentary: Yes, this is mediocre.")